# Assignment 3

#### Student ID: *Double click here to fill the Student ID*

#### Name: *Double click here to fill the name*

## Q1: Analyze StackOverflow dataset using SQL

Kaggle has a rich number of [BigQuery](https://www.kaggle.com/datasets?fileType=bigQuery) and [SQLite](https://www.kaggle.com/datasets?fileType=sqlite) datasets that you can practice your SQL skill.

In this question, we are going to examine the StackOverflow dataset. [Stack Overflow](https://stackoverflow.com/) is a popular question and answer site for technical questions. We will explore how to set up a service that identifies the Stack Overflow users who have demonstrated expertise within a specific field by answering related questions about it.

Hint: It is recommended to answer this question in Kaggle, where you can access the dataset directly. In addition, refer to https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf and our laboratory for useful commands.

Firstly, if you are using colab, use the following code snippet to setup the client. For more detail, please refer to our lab.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Replace the following variable your key path 
key_path = "utopian-datum-340514-9ffc23108bf4.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

(a) Try to print the names of all tables available in the dataset first. 
Then, try to print the table schema of the `posts_answers` table and the `posts_questions` tables. Finally, preview the first ten rows of the above two tables by retrieving the data frame.

Hint: the dataset can be access via `dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")`

In [ ]:
# coding your answer here.

(b) `posts_questions` has a column called tags which lists the topics/technologies each question is about. Please write a query that selects the `id`, `title`, `parent_id` and `owner_display_name` columns from the `posts_questions` table and follow the below restrictions. 

* Restrict the results to rows that contain the word 'sql' in the `tags` column. (Remember the `SELET...WHERE` clause. Do not use wildcard here.)
* Rename the column name `title` to `question_title` (Remember the `AS` clause)
* Sort the results by the `score` column in descending order (Remember the `ORDER BY` clause)

In the retrieve data frame, how many rows do you have? How many rows have non-null `ower_dispaly_name`? (Which means the `owner_display_name` is not 'None').

In [ ]:
# coding your answer here.
query = """
                  """

############   Do not modify the code below     ############ 
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############ 

(c) `posts_answers` has a column called `parent_id` which identifies the ID of the question each answer corresponds to. You can then join two tables by the `parent_id` in `posts_answers` and the `id` in `posts_questions`. `posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question. 

Now, write a query that has a single row for each user who answered at least one question with a tag that equals to string `sql`. Your results should have two columns (Remember the `SELET...WHERE` clause) and follow the below restrictions:

* `owner_user_id` - contains the `owner_user_id` column from the `posts_answers` table (Remember the `INNER JOIN` clause)
* `number_of_answers` - contains the number of answers the user has written to `sql` related questions (Remember the `AS`, `GROUP BY` and `COUNT` clause)
* Filter out the rows whose `number_of_answers` is smaller or equal to 2 (Remember the `HAVING` clause)

In the retrieve data frame, which user answers most questions in the sql domain? The user is an expert in the SQL domain and is the author of many SQL books, try to find the name of a book the user wrote.

Hint: You can find the public user profile by appending the ID after https://stackoverflow.com/users/

In [ ]:
# coding your answer here.
query = """
                         """

############   Do not modify the code below     ############ 
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############ 

## Q2: Data cleaning with Melbourne Housing dataset

In this question, you are going to practice the data preparation skill that is often used in a real-world project.

The dataset is a snapshot of a [dataset](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market) created by Tony Pino. It was scraped from publicly available results posted every week from https://www.domain.com.au/. The dataset includes Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale and distance from the central business district, etc.

Firstly, use the following code snippet to set up the dataset. Note the CSV file can be downloaded from our course website.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function to calculate mae by fixing the model
def scoring_mae(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=20, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

data = pd.read_csv('melb_data.csv')
y = data.Price

# To keep things simple, we'll split the columns into numerical can categorical features
melb_predictors = data.drop(['Price', 'Date', 'Address'], axis=1)
cat_col = melb_predictors.select_dtypes(exclude=['int64','float64'])

# Divide data into training and validation subsets, try to use the resulting datafram below in the following questions
X, X_v, y_train, y_valid = train_test_split(melb_predictors, y, train_size=0.8, test_size=0.2, random_state=0)
# Numerical columns
X_train = X.select_dtypes(exclude=['object'])
X_valid = X_v.select_dtypes(exclude=['object'])
# Categorical columns
X_train_cat = X.select_dtypes(exclude=['int64','float64'])
X_valid_cat = X_v.select_dtypes(exclude=['int64','float64'])

(a) Firstly, try to find out which columns contain missing values. Then, try to calculate the percentage of the missing values in the dataset.  

Hint: You should count the missing rate base on the original `data` matrix

In [ ]:
# coding your answer here.

(b) Considering only the numerical columns in this question. We are going to compare the MAE between the three data cleaning approach.

1. Removing all the columns with empty values. 
2. Replace missing values with the median value along each column.
3. Use the iterative imputation method and set the regressor to KNN regressor with 20 neighbors.

Use the `scoring_mae` function to perform regression and calculate the MAE for each approach. Finally, make some comments on the results.

Hint: Since we are working with both training and validation sets, try to drop the same columns in both data frames. In addition, you should apply the same transform when you impute the missing value.

In [ ]:
# coding your answer here.

(c) Considering only the categorical columns in this question. Find out which columns contain missing values first. Then, try to 

* Replace missing values with the most frequent value along each column. 
* Perform one-hot encoding for the categorical variables whose number of categories is smaller than 10; otherwise, use the label encoding for that column. 
* When there are unknown categories in the validation set, set it to all zeros for the one-hot encoding and set it to -1 for the label encoding.

Use the `scoring_mae` function to perform regression and calculate the MAE for the resulting data.

Hint: Since we are working with both training and validation sets, try to apply the same transform when you impute the missing data or encode the variables. You may find [ColumnTransformer](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html) and the `handle_unknown` option in the encoder useful.

In [ ]:
# coding your answer here.

(d) Combining the transformation in (b) and (c) to the original training and validation data split (`X` and `X_v`) and using the `scoring_mae` function to calculate the MAE. Make some comments on the results when comparing with (b) and (c).

Hint: Since we are working with both training and validation sets, try to apply the same transform when you impute the missing data or encode the variables. You may find [ColumnTransformer](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html) useful.

In [ ]:
# coding your answer here.

## Q3: Feature engineering and selection with Ames housing dataset

In this question, we are going to examine several feature engineering and feature selection methods.

The dataset we are going to use is a modified version of the Ames housing dataset. The original data was compiled by Dean De Cock for use in data science education and published in [De Cock, D. (2011)](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The modified version contains 2930 rows with 79 columns describing every aspect of residential homes in Ames, Iowa.  

Firstly, use the following code snippet to set up the dataset. Note the CSV file can be downloaded from our course website.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Notice that the categorical variables in your dataset will be label encoded 
# and change the original value when you call the following functions 
def scoring_rmsle(X, y, model=RandomForestRegressor()):
    # Label encoding for categorical variables
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

# Prepare data
df = pd.read_csv("ames.csv")
X = df.copy()
y = X.pop("SalePrice")

(a) First try to perform the k-means clustering with the following parameters:
* features: `FirstFlrSF`, `SecondFlrSF`,`GrLivArea`, `LotArea`, `TotalBsmtSF` 
* number of clusters: 10

Then, add the k-means label and cluster-distances features to your original dataset. Finally, perform the regression and calculate the RMLSE using the `scoring_rmsle` function.

Hint: You may find the `predict` and `transform` in the [k-means function](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) useful when you are generating the features. 

In [ ]:
# coding your answer here.

(b) Try to perform the permutation importance with the regressor set to extra-trees. Draw the bar plot of the feature importance of the top 10 most important features. Finally, use the most important 50 features to perform regression and calculate the RMLSE using the `scoring_rmsle` function.

Hint: The categorical variables should be label encoded when calculating permutation importance.

In [ ]:
# coding your answer here.

(c) Use the [Boruta](https://pdfs.semanticscholar.org/ecc2/ca3150dc4d4d8dceedab244114f191e05742.pdf) to perform feature selection. How many features does Boruta select in this example? Then, use the selected features to perform regression and calculate the RMLSE using the `scoring_rmsle` function. Finally, make some comments comparing with (a) and (b).

Hint: It may take some time to perform the Boruta. The categorical variables should be label encoded when using Boruta.

In [ ]:
# coding your answer here.